## Caching Variables on Disk

*[Coding along with the Udemy Course [Advanced Retrieval Augmented Generation ](https://www.udemy.com/course/advanced-retrieval-augmented-generation/) by Rémi Connesson]*

In [1]:
import pandas as pd
from openai import AsyncOpenAI

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [3]:
client = AsyncOpenAI(api_key=api_key)

In [4]:
def _msg(role, content):
    return {'role': role, 'content': content}

def system(content):
    return _msg('system', content)

def user(content):
    return _msg('user', content)

def assistant(content):
    return _msg('assistant', content)

In [5]:
model = "gpt-4o-mini"

In [9]:
# sanity check 1, is this thing on?
completion_1 = await client.chat.completions.create(
    model = model,
    messages = [user("What is caching in software engineering?")],
    max_tokens = 100 # limit the output to save costs; answer might get cut
)
completion_1.choices[0].message.content

'Caching in software engineering is a technique used to store copies of frequently accessed data or computations in a temporary storage location, known as a cache, to improve the performance and efficiency of applications by reducing latency and resource consumption. The primary purpose of caching is to speed up data retrieval and decrease the load on underlying data sources or computation resources.\n\n### Key Concepts of Caching:\n\n1. **Cache Types**:\n   - **Memory Cache**: In-memory caches, such as Redis or Memcached, store data'

In [11]:
# sanity check 2, is this thing on?
completion_2 = await client.chat.completions.create(
    model = "gpt-4o-mini-2024-07-18",
    messages = [user("What date is today?")]
)
completion_2.choices[0].message.content

"Today's date is October 3, 2023."

### Introducing Caching

In [17]:
# https://pypi.org/project/diskcache/
# Disk Cache -- Disk and file backed persistent cache
from diskcache import Cache

In [18]:
cache = Cache(directory=".cache")

In [21]:
# cache.set("thirteen", "It was a bright cold day in April, and the clocks were striking thirteen.")

In [22]:
cache.get("thirteen")

'It was a bright cold day in April, and the clocks were striking thirteen.'

### Making Caching Asynchronous

In [23]:
import asyncio

In [27]:
# creating a wrapper around the cache
# so I can call it in a way that's thread safe
async def set_async(key, val, **kwargs): # what the hell is kwargs???
    # await the cache.set operation
    return await asyncio.to_thread(cache.set, key, val, **kwargs)

async def get_async(key, default=None, **kwargs):
    return await asyncio.to_thread(cache.get, key, default, **kwargs)

In [28]:
await get_async("thirteen")

'It was a bright cold day in April, and the clocks were striking thirteen.'

In [30]:
print(await get_async("key that doesn't exist", default="NOT FOUND")) # returns None if default is not set

NOT FOUND


In [32]:
(
    await get_async("key that doesn't exist", default="NOT FOUND"),
    await set_async("Cooper", "The Owls Are Not What They Seem"),
    await get_async("Cooper")
)

('NOT FOUND', True, 'The Owls Are Not What They Seem')

#### <span style="color:green;font-weight:bold;font-size:105%">Interlude: What are **kwargs when calling a Python function?</span>

In Python, `**kwargs` is a syntax used in function definitions to allow passing a variable number of keyword arguments to a function. The name `kwargs` stands for "keyword arguments," though you can name it anything you like (the double asterisks `**` are what matter). 

Here’s how it works:

1. **Collecting Keyword Arguments**: When `**kwargs` is used in a function definition, it collects all extra keyword arguments passed to the function into a dictionary. The keys in the dictionary are the argument names, and the values are the values passed for those arguments.

   ```python
   def example_function(**kwargs):
       print(kwargs)

   example_function(name="Alice", age=25, job="Data Scientist")
   # Output: {'name': 'Alice', 'age': 25, 'job': 'Data Scientist'}
   ```

2. **Using `**kwargs` to Flexibly Pass Arguments**: This is useful for functions that might take optional parameters or need to handle a variety of inputs without explicitly defining each parameter.

3. **Unpacking Keyword Arguments**: You can also use `**kwargs` when calling a function to "unpack" a dictionary as keyword arguments.

   ```python
   def greet(name, greeting):
       print(f"{greeting}, {name}!")

   details = {"name": "Alice", "greeting": "Hello"}
   greet(**details)
   # Output: "Hello, Alice!"
   ```

Overall, `**kwargs` provides flexibility, especially for writing functions that can handle optional or dynamically named arguments.

#### <span style="color:green;font-weight:bold;font-size:105%">Interlude End</span>

### Caching LLM Calls

In [37]:
import json
from hashlib import md5

In [38]:
# when calling a function in Python there are different ways to arrange the oder of the arguments
# we've to make sure that arguments are in a certain order when he create a hashkey out of them
def make_cache_key(key_name, **kwargs):
    kwargs_string = json.dumps(kwargs, sort_keys=True)
    kwargs_hash = md5(kwargs_string.encode('utf-8')).hexdigest()
    cache_key = f"{key_name}__{kwargs_hash}"
    return cache_key

In [39]:
make_cache_key("demo_cache", a=1, b=2, c=4)

'demo_cache__ac6b59f8b9221cc50603ef2f4fcbf866'

In [40]:
make_cache_key("demo_cache", a=1, c=4, b=2)

'demo_cache__ac6b59f8b9221cc50603ef2f4fcbf866'

#### __Caching an LLM Call__

In [ ]:
# caching a chat completion
# the * at the position of the first parameter forces us to explicitly pass parameters with the variable name
# Positional-Only Arguments: When you use a single * by itself in the function signature, it indicates that all arguments 
# following the * must be passed as keyword arguments. This is useful for enforcing readability, as it makes certain 
# arguments require a name when the function is called.
def _make_cache_key_for_chat_completion(
    *,
    model,
    messages,
    **kwargs
):
    return make_cache_key(
        "openai_chat_completion",
        model=model,
        messages=messages,
        **kwargs
    )

In [43]:
# A sentinel value is often used in function parameters to indicate that no value was provided by the user. 
# A common sentinel for this purpose is None.
CACHE_MISS_SENTINEL = object() # creating a sentinel

In [ ]:
async def cached_chat_completion(
    model
    messages,
    **kwargs
):
    # 1) CREATE CACHE KEY
    cache_key = _make_cache_key_for_chat_completion(
        *,
        model,
        messages,
        **kwargs
    )

    cached_value = await get_async(cache_key)

    # 2) CACHE HIT
    

    # 3) CACHE MISS